In [1]:
from utils.generator import Generator
from utils.mean_subtract import MeanSubtract

from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense

from keras.optimizers import Adam
from keras.callbacks import TensorBoard

import pickle

Using TensorFlow backend.
/home/yq/miniconda3/envs/tf/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/yq/miniconda3/envs/tf/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
class SmallerVGGNet:
    @staticmethod
    def build(width, height, depth):
        model = Sequential()
        inputShape = (height, width, depth)
        
        model.add(Conv2D(32, 3, padding="same", input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=-1))
        model.add(MaxPooling2D(pool_size=3))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(64, 3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=-1))
        model.add(Conv2D(64, 3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=-1))
        model.add(MaxPooling2D(pool_size=2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(128, 3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=-1))
        model.add(Conv2D(128, 3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=-1))
        model.add(MaxPooling2D(pool_size=2))
        model.add(Dropout(0.25))
        
        model.add(Flatten())
        model.add(Dense(1024))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=-1))
        model.add(Dropout(0.5))
        
        model.add(Dense(4))
        
        return model

In [3]:
width = 96
height = 96
depth = 3
BS = 128
epochs = 100

In [4]:
model = SmallerVGGNet.build(width, height, depth)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 96, 96, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 96, 96, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 96, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 64)        0         
__________

### Training on extra dataset (pretrain)

In [5]:
means = pickle.load(open("rgb_means.pkl", "rb"))
ms = MeanSubtract(means["R"], means["G"], means["B"])
preprocessors = [ms]

generator = Generator("bbox/extra.hdf5", BS, epochs, preprocessors)

In [6]:
opt = Adam(lr=0.001)
model.compile(loss="mean_squared_error", optimizer=opt)

In [7]:
H = model.fit_generator(generator.generate(), steps_per_epoch=generator.n_img//BS, epochs=20)

Epoch 1/20
1580/1580 [==============================] - 212s 134ms/step - loss: 439.3331
Epoch 2/20
1580/1580 [==============================] - 209s 132ms/step - loss: 136.8864
Epoch 3/20
1580/1580 [==============================] - 209s 132ms/step - loss: 125.4200
Epoch 4/20
1580/1580 [==============================] - 209s 132ms/step - loss: 117.0011
Epoch 5/20
1580/1580 [==============================] - 210s 133ms/step - loss: 112.8997
Epoch 6/20
1580/1580 [==============================] - 210s 133ms/step - loss: 110.4777
Epoch 7/20
1580/1580 [==============================] - 210s 133ms/step - loss: 107.3164
Epoch 8/20
1580/1580 [==============================] - 210s 133ms/step - loss: 105.3305
Epoch 9/20
1580/1580 [==============================] - 209s 132ms/step - loss: 103.3198
Epoch 10/20
1580/1580 [==============================] - 209s 132ms/step - loss: 101.5092
Epoch 11/20
1580/1580 [==============================] - 209s 132ms/step - loss: 100.6882
Epoch 12/20
1580/15

### Training on training set

In [8]:
import numpy as np

def exp_decay(epoch):
    initial_lrate = 0.001
    k = 0.1
    lrate = initial_lrate * np.exp(-k*epoch)
    
    return lrate

In [9]:
from keras.callbacks import LearningRateScheduler

train_gen = Generator("bbox/train.hdf5", BS, epochs, preprocessors)
val_gen = Generator("bbox/val.hdf5", BS, epochs, preprocessors)

callbacks = [LearningRateScheduler(exp_decay)]

In [10]:
H = model.fit_generator(train_gen.generate(), validation_data=val_gen.generate(), 
                        steps_per_epoch=train_gen.n_img//BS, validation_steps=val_gen.n_img//BS,
                       epochs=epochs, callbacks=callbacks)

Epoch 1/100
241/241 [==============================] - 11s 44ms/step - loss: 142.6494 - val_loss: 126.0216
Epoch 2/100
241/241 [==============================] - 10s 44ms/step - loss: 115.8356 - val_loss: 135.4119
Epoch 3/100
241/241 [==============================] - 10s 43ms/step - loss: 109.5497 - val_loss: 103.2265
Epoch 4/100
241/241 [==============================] - 10s 42ms/step - loss: 102.5558 - val_loss: 104.2504
Epoch 5/100
241/241 [==============================] - 10s 43ms/step - loss: 97.1273 - val_loss: 104.1343
Epoch 6/100
241/241 [==============================] - 10s 43ms/step - loss: 93.5825 - val_loss: 105.5293
Epoch 7/100
241/241 [==============================] - 10s 43ms/step - loss: 89.6558 - val_loss: 101.8422
Epoch 8/100
241/241 [==============================] - 10s 43ms/step - loss: 88.2924 - val_loss: 99.5892
Epoch 9/100
241/241 [==============================] - 11s 44ms/step - loss: 85.2411 - val_loss: 97.6035
Epoch 10/100
241/241 [======================

Epoch 78/100
241/241 [==============================] - 10s 43ms/step - loss: 65.4867 - val_loss: 98.6388
Epoch 79/100
241/241 [==============================] - 11s 44ms/step - loss: 65.5711 - val_loss: 100.7474
Epoch 80/100
241/241 [==============================] - 10s 43ms/step - loss: 64.7837 - val_loss: 99.0439
Epoch 81/100
241/241 [==============================] - 11s 44ms/step - loss: 65.8228 - val_loss: 99.5532
Epoch 82/100
241/241 [==============================] - 10s 44ms/step - loss: 65.4032 - val_loss: 99.0749
Epoch 83/100
241/241 [==============================] - 10s 44ms/step - loss: 65.1921 - val_loss: 96.8132
Epoch 84/100
241/241 [==============================] - 10s 43ms/step - loss: 65.3921 - val_loss: 99.1173
Epoch 85/100
241/241 [==============================] - 11s 44ms/step - loss: 65.2443 - val_loss: 99.5760
Epoch 86/100
241/241 [==============================] - 11s 44ms/step - loss: 65.6453 - val_loss: 100.9758
Epoch 87/100
241/241 [======================

In [11]:
model.save("detection.h5")